In [76]:
import tensorflow as tf 
from tensorflow.keras import layers, models
import pandas as pd
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [7]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
gpu = tf.config.list_physical_devices('GPU')
if gpu:
  # only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpu[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpu), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    print(e)

1 Physical GPUs, 1 Logical GPU


In [18]:
#from feature_extraction import flatten_ts

data = pd.read_csv("data3.csv")
# flat_data = flatten_ts(data)
# flat_data
data

,xAcc,yAcc,zAcc,xGyro,yGyro,zGyro,label
0,6.99,-0.57,-7.28,-2.75,-3.23,2.62,fall
1,6.51,-0.75,-6.93,-0.67,-6.35,4.64,fall
2,6.22,-0.63,-6.73,0.79,-5.49,3.85,fall
3,6.34,-0.62,-6.80,1.59,-2.26,0.67,fall
4,6.49,-0.39,-6.60,0.67,-0.24,-1.10,fall
...,...,...,...,...,...,...,...
96795,7.30,-1.16,-4.58,18.19,3.60,-35.77,light
96796,7.39,-0.37,-5.05,20.08,0.06,-34.55,light
96797,7.52,-1.46,-5.82,22.58,0.12,-28.02,light
96798,7.58,-2.14,-6.31,19.47,2.44,-22.52,light


In [28]:
new_data = data.drop("label",axis=1).to_numpy().reshape(-1, 400, 6)
labels = np.array(data.label.iloc[np.arange(0,data.shape[0], 400)])

In [71]:
X_train, X_test, y_train, y_test = train_test_split(new_data, labels)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train.astype(np.float64))
# X_test = scaler.fit_transform(X_test.astype(np.float64))

le = LabelEncoder()
le.fit(np.unique(y_train))
y_train = le.transform(y_train)
y_test = le.transform(y_test)
y_train = tf.keras.utils.to_categorical(y_train, num_classes=7)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=7)

seq_len, n_features = X_train.shape[1:]

X_train = X_train.reshape(-1, seq_len, n_features) 
X_test = X_test.reshape(-1, seq_len, n_features)

In [63]:
X_train.shape

(181, 400, 6)

# First CNN

In [72]:
first_model = models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(seq_len, n_features)),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(7, activation='sigmoid')  # Output layer for binary classification
])

In [73]:
first_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

In [74]:
first_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 398, 32)           608       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 199, 32)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 6368)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                407616    
_________________________________________________________________
dense_11 (Dense)             (None, 7)                 455       
Total params: 408,679
Trainable params: 408,679
Non-trainable params: 0
_________________________________________________________________


In [81]:
first_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
6/6 [==============================] - 0s 43ms/step - loss: 0.9238 - accuracy: 0.9116 - val_loss: 1.9449 - val_accuracy: 0.7213
Epoch 2/10
6/6 [==============================] - 0s 11ms/step - loss: 0.3300 - accuracy: 0.9392 - val_loss: 1.6764 - val_accuracy: 0.7541
Epoch 3/10
6/6 [==============================] - 0s 12ms/step - loss: 0.2489 - accuracy: 0.9558 - val_loss: 1.5600 - val_accuracy: 0.6721
Epoch 4/10
6/6 [==============================] - 0s 10ms/step - loss: 0.0653 - accuracy: 0.9834 - val_loss: 1.4410 - val_accuracy: 0.7869
Epoch 5/10
6/6 [==============================] - 0s 9ms/step - loss: 0.0249 - accuracy: 0.9945 - val_loss: 1.3878 - val_accuracy: 0.7541
Epoch 6/10
6/6 [==============================] - 0s 11ms/step - loss: 0.0147 - accuracy: 1.0000 - val_loss: 1.4257 - val_accuracy: 0.7541
Epoch 7/10
6/6 [==============================] - 0s 10ms/step - loss: 0.0078 - accuracy: 1.0000 - val_loss: 1.4809 - val_accuracy: 0.7213
Epoch 8/10
6/6 [============

In [82]:
first_model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 1.5640 - accuracy: 0.7213


[1.5639760494232178, 0.7213114500045776]